<a href="https://colab.research.google.com/github/Kgs-Mathaba/Apple-Stock-Price-Prediction-And-Forecasting-Using-LSTM-Neural-Network/blob/main/Apple_Stock_Price_Prediction_And_Forecasting_Using_LSTM_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Data Collection
import pandas_datareader as pdr

df = pdr.get_data_tiingo('AAPL', api_key = key)
df.to_csv('AAPL.csv')

In [3]:
import pandas as pd
df = pd.read_csv('AAPL.csv')
df.head(10)

,symbol,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
0,AAPL,2016-12-23 00:00:00+00:00,116.52,116.5200,115.59,115.59,14249484,27.416247,27.416247,27.197425,27.197425,56997936,0.0,1.0
1,AAPL,2016-12-27 00:00:00+00:00,117.26,117.8000,116.49,116.52,18296855,27.590363,27.717421,27.409188,27.416247,73187420,0.0,1.0
2,AAPL,2016-12-28 00:00:00+00:00,116.76,118.0166,116.20,117.52,20905892,27.472717,27.768385,27.340953,27.651539,83623568,0.0,1.0
3,AAPL,2016-12-29 00:00:00+00:00,116.73,117.1095,116.40,116.45,15039519,27.465658,27.554951,27.388012,27.399776,60158076,0.0,1.0
4,AAPL,2016-12-30 00:00:00+00:00,115.82,117.2000,115.43,116.65,30586265,27.251542,27.576245,27.159778,27.446835,122345060,0.0,1.0
5,AAPL,2017-01-03 00:00:00+00:00,116.15,116.3300,114.76,115.80,28781865,27.329189,27.371541,27.002132,27.246836,115127460,0.0,1.0
6,AAPL,2017-01-04 00:00:00+00:00,116.02,116.5100,115.75,115.85,21118116,27.298601,27.413894,27.235072,27.258601,84472464,0.0,1.0
7,AAPL,2017-01-05 00:00:00+00:00,116.61,116.8642,115.81,115.92,22193587,27.437423,27.497234,27.249189,27.275071,88774348,0.0,1.0
8,AAPL,2017-01-06 00:00:00+00:00,117.91,118.1600,116.47,116.78,31751900,27.743303,27.802126,27.404482,27.477423,127007600,0.0,1.0
9,AAPL,2017-01-09 00:00:00+00:00,118.99,119.4300,117.94,117.95,33561948,27.997418,28.100947,27.750362,27.752715,134247792,0.0,1.0
